# 🧠 Clasificación de la Enfermedad de Alzheimer
Integrantes
- Diego Alexander Hernández Silvestre - 21270
- Linda Inés Jiménez Vides - 21169
- Mario Antonio Guerra Morales - 21008
- Kristopher Javier Alvarado López - 21188

In [1]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import random
import shutil

#### 📊 Balanceo de Data

In [2]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# Definir las transformaciones que aplicarás
augmentations = transforms.Compose([
    transforms.RandomRotation(degrees=10),  # Rotación de ±10 grados
    transforms.RandomAffine(degrees=0, translate=(0.02, 0.02)),  # Desplazamiento horizontal/vertical 2%
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.92, 1.08)),  # Zoom hasta 8%
    transforms.ToTensor(),
])

# Clase personalizada para cargar imágenes desde carpetas
class DementiaDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_list = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_list[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Directorios de las carpetas de imágenes
folders = {
    "MildDemented": "data/train/MildDemented",
    "ModerateDemented": "data/train/ModerateDemented",
    "NonDemented": "data/train/NonDemented",
    "VeryMildDemented": "data/train/VeryMildDemented"
}

# Directorios donde se guardarán las imágenes aumentadas
augmented_folders = {
    "MildDemented": "data/augmented/MildDemented",
    "ModerateDemented": "data/augmented/ModerateDemented",
    "NonDemented": "data/augmented/NonDemented",
    "VeryMildDemented": "data/augmented/VeryMildDemented"
}

target_size = 3200  # Número objetivo de imágenes por clase

# Crear los directorios para las imágenes aumentadas si no existen
for folder in augmented_folders.values():
    if not os.path.exists(folder):
        os.makedirs(folder)

# Función para aplicar augmentación y guardar nuevas imágenes
def augment_and_save(dataset, save_dir, target_size):
    current_size = len(dataset)
    images_to_generate = target_size  # Calcular cuántas imágenes generar para llegar a 3200
    counter = 0  # Contador para las nuevas imágenes aumentadas
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    # Generar imágenes hasta alcanzar el tamaño objetivo
    while counter < images_to_generate:
        for batch in loader:
            augmented_img = transforms.ToPILImage()(batch[0])  # Convertir tensor a imagen PIL
            augmented_img.save(os.path.join(save_dir, f'augmented_{counter + current_size}.jpg'))  # Guardar imagen aumentada
            counter += 1
            if counter >= images_to_generate:
                break

# Iterar sobre cada carpeta de imágenes originales
for label, folder in folders.items():
    current_images = os.listdir(folder)
    current_size = len(current_images)
        
    # Si el tamaño actual es menor a 3200, se realiza augmentación
    if current_size < target_size:
        print(f"Aplicando augmentación en {label}. Tamaño actual: {current_size}.")
        
        # Cargar el dataset de la clase actual
        dataset = DementiaDataset(image_dir=folder, transform=augmentations)
        
        
        # Definir el directorio para guardar imágenes aumentadas
        save_dir = augmented_folders[label]
        
        # Aumentar imágenes y guardar en la carpeta de imágenes aumentadas
        augment_and_save(dataset, save_dir, target_size)
        
    else:
        print(f"No se necesita augmentación en {label}. Tamaño actual: {current_size}.")


Aplicando augmentación en MildDemented. Tamaño actual: 717.
Aplicando augmentación en ModerateDemented. Tamaño actual: 52.
Aplicando augmentación en NonDemented. Tamaño actual: 2560.
Aplicando augmentación en VeryMildDemented. Tamaño actual: 1792.


In [3]:
for label, folder in augmented_folders.items():
    current_images = os.listdir(folder)
    current_size = len(current_images)
    print(f"Clase: {label}. Tamaño actual: {current_size}.")

Clase: MildDemented. Tamaño actual: 3200.
Clase: ModerateDemented. Tamaño actual: 3200.
Clase: NonDemented. Tamaño actual: 3200.
Clase: VeryMildDemented. Tamaño actual: 3200.


#### 🏋🏽‍♀️ División entrenamiento y validación

In [4]:
# Directorios donde se guardarán las imágenes divididas
train_folder = "data/augmented/"

train_dir = "new_data/train/"
val_dir = "new_data/validation/"

# Crear los directorios de train y validation si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Contadores para el total de imágenes en train y validation
total_train_images = 0
total_val_images = 0

# Función para dividir y copiar las imágenes de train en train/validation
def split_train_validation(train_folder, label, train_dir, val_dir, split_ratio=0.7):
    global total_train_images, total_val_images
    images = os.listdir(os.path.join(train_folder, label))
    random.shuffle(images)
    
    # Calcular cuántas imágenes irán a train y cuántas a validation
    split_index = int(len(images) * split_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]
    
    # Actualizar los contadores
    total_train_images += len(train_images)
    total_val_images += len(val_images)
    
    # Crear carpetas de train y validation para la clase actual
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    
    # Copiar imágenes de train
    for img in train_images:
        shutil.copy(os.path.join(train_folder, label, img), os.path.join(train_dir, label, img))
    
    # Copiar imágenes de validation
    for img in val_images:
        shutil.copy(os.path.join(train_folder, label, img), os.path.join(val_dir, label, img))

# Iterar sobre las carpetas de cada clase dentro de train
for label in os.listdir(train_folder):
    split_train_validation(train_folder, label, train_dir, val_dir, split_ratio=0.7)

# Imprimir el total de imágenes en train y validation
print(f"Total de imágenes en train: {total_train_images}")
print(f"Total de imágenes en validation: {total_val_images}")
print("División de imágenes de train en train/validation completada.")

Total de imágenes en train: 8960
Total de imágenes en validation: 3840
División de imágenes de train en train/validation completada.


#### Arquitectura del Modelo

In [5]:
# Configuración del dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformaciones (Aumento de datos y normalización)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar las imágenes
    transforms.RandomHorizontalFlip(),  # Voltear horizontalmente
    transforms.RandomRotation(10),  # Rotar aleatoriamente hasta 10 grados
    transforms.ToTensor(),  # Convertir a tensor (esto convierte automáticamente a float y escala a [0, 1])
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalización estándar
])

In [6]:
# Cargar los datasets de entrenamiento y validación
train_data = datasets.ImageFolder('new_data/train', transform=transform)
val_data = datasets.ImageFolder('new_data/validation', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [7]:
# Definición del modelo CNN basado en la arquitectura del artículo
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Entrada 224x224x3 -> Salida 224x224x32
        self.pool = nn.MaxPool2d(2, 2)  # Reduce 2x cada dimensión: 112x112x32 después de la 1ra convolución
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Entrada 112x112x32 -> Salida 112x112x64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Entrada 56x56x64 -> Salida 56x56x128
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Capa totalmente conectada, entrada 128*28*28 = 100352, salida 512
        self.fc2 = nn.Linear(512, 4)  # Capa de salida, 4 clases para la clasificación
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + MaxPool
        x = self.pool(F.relu(self.conv3(x)))  # Conv3 + MaxPool
        x = x.view(-1, 128 * 28 * 28)  # Aplanar para la capa totalmente conectada
        x = F.relu(self.fc1(x))  # FC1
        x = self.dropout(x)  # Dropout
        x = self.fc2(x)  # FC2 - Salida final
        return x

#### Entrenamiento del Modelo

In [8]:
# Inicializar el modelo
model = CNNModel().to(device)

# Usar CrossEntropyLoss, que combina softmax y entropía cruzada
criterion = nn.CrossEntropyLoss()

# Definir el optimizador
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Entrenamiento del modelo
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Empieza el entrenamiento para la época {epoch+1} \n")
        model.train()  # Modo de entrenamiento
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Limpiar gradientes previos
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Calcular gradientes
            optimizer.step()  # Actualizar pesos

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print("Pérdida por época calculada.\n")

        # Validación
        model.eval()  # Modo de evaluación (sin actualización de gradientes)
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        print("Pérdida por validación calculada.\n")
        val_loss = val_loss / len(val_loader.dataset)
        accuracy = correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}')

    print('Entrenamiento completo.')

In [9]:
# Entrenar el modelo
train_model(model, train_loader, val_loader, criterion, optimizer)

# Guardar el modelo
torch.save(model.state_dict(), 'model.pth')

Empieza el entrenamiento para la época 1 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [1/10], Loss: 1.2530, Val Loss: 0.9360, Val Accuracy: 0.5751
Empieza el entrenamiento para la época 2 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [2/10], Loss: 0.8672, Val Loss: 0.7747, Val Accuracy: 0.6434
Empieza el entrenamiento para la época 3 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [3/10], Loss: 0.7621, Val Loss: 0.6823, Val Accuracy: 0.6849
Empieza el entrenamiento para la época 4 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [4/10], Loss: 0.7006, Val Loss: 0.6425, Val Accuracy: 0.7134
Empieza el entrenamiento para la época 5 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [5/10], Loss: 0.6370, Val Loss: 0.5898, Val Accuracy: 0.7324
Empieza el entrenamiento para la época 6 

Pérdida por época calculada.

Pérdida por validación calculada.

Epoch [6/10], Los

In [10]:
# Evaluar el modelo en los datos de validación
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    print(f'Precisión en el conjunto de validación: {accuracy * 100:.2f}%')

# Crear una instancia del modelo y cargar el estado guardado
model = CNNModel().to(device)
model.load_state_dict(torch.load('model.pth'))

# Evaluar el modelo
evaluate_model(model, val_loader)

Precisión en el conjunto de validación: 78.34%


In [11]:
import shap

# Inicializar el explainer usando una pequeña muestra de tu conjunto de entrenamiento
# Usamos una pequeña muestra para mejorar el rendimiento
sample_data = next(iter(train_loader))[0][:100].to(device)  # Ejemplo con 100 imágenes del conjunto de entrenamiento

# Redimensionar sample_data para que coincida con la entrada del modelo
sample_data = sample_data.view(-1, 3, 224, 224)

# Inicializar el SHAP explainer
explainer = shap.DeepExplainer(model, sample_data)

# Obtención de los SHAP values para un lote de imágenes del conjunto de validación
inputs, _ = next(iter(val_loader))
inputs = inputs.to(device)

# Redimensionar inputs para que coincida con la entrada del modelo
inputs = inputs.view(-1, 3, 224, 224)

shap_values = explainer.shap_values(inputs)


c:\Users\Alvar\OneDrive\Documentos\SemestreVIII\Deep Learning\Classification_of_Alzheimers_Disease\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: The size of tensor a (56) must match the size of tensor b (28) at non-singleton dimension 3

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import shap
import numpy as np
import matplotlib.pyplot as plt

# 1. Definir el modelo CNN
class AlzheimerCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(AlzheimerCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)  # Usamos ResNet18 como base
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Inicializar el modelo, la función de pérdida y el optimizador
num_classes = 4
model = AlzheimerCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 2. Preparación de los datos usando ImageFolder
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Cargar el dataset de entrenamiento usando ImageFolder
train_dataset = ImageFolder(root='data/augmented', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Cargar el dataset de prueba usando ImageFolder
# Asegúrate de tener un conjunto de prueba organizado en carpetas por clase
test_dataset = ImageFolder(root='data/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Época [{epoch + 1}/{num_epochs}], Pérdida: {running_loss / len(train_loader):.4f}")

# 4. Evaluación en conjunto de prueba
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Precisión del modelo en el conjunto de prueba: {100 * correct / total:.2f}%")

# 5. Cálculo y visualización de valores SHAP
# Selecciona algunas imágenes del conjunto de entrenamiento para el cálculo de SHAP
sample_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
images, labels = next(iter(sample_loader))

# Cargar valores SHAP usando DeepExplainer
explainer = shap.DeepExplainer(model, images)
shap_values = explainer.shap_values(images)

# Visualización de los valores SHAP
for i in range(len(shap_values)):
    plt.figure()
    shap.image_plot([shap_values[i]], images.numpy().transpose(0, 2, 3, 1))


c:\Users\Alvar\OneDrive\Documentos\SemestreVIII\Deep Learning\Classification_of_Alzheimers_Disease\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Alvar\OneDrive\Documentos\SemestreVIII\Deep Learning\Classification_of_Alzheimers_Disease\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Época [1/10], Pérdida: 0.5928
Época [2/10], Pérdida: 0.3427
Época [3/10], Pérdida: 0.1968
Época [4/10], Pérdida: 0.1362
Época [5/10], Pérdida: 0.0773
Época [6/10], Pérdida: 0.0775
Época [7/10], Pérdida: 0.0555
Época [8/10], Pérdida: 0.0725
Época [9/10], Pérdida: 0.0447
Época [10/10], Pérdida: 0.0402
Precisión del modelo en el conjunto de prueba: 67.47%


RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.

In [13]:
# Guardar el modelo
torch.save(model.state_dict(), 'modelo_alzheimer.pth')
print("Modelo guardado exitosamente.")


Modelo guardado exitosamente.


In [37]:
import shap

model.eval()



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(64, 64, kernel_s